In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp errors

In [ ]:
#| export
import numpy as np
import math
from abc import ABC, abstractmethod


In [ ]:
#| export
from pct.functions import IndexedParameter
from pct.putils import smooth
# from pct.hierarchy import PCTHierarchy


In [ ]:
from unittest import TestCase

# ErrorResponse - aggregation

## BaseErrorType

In [ ]:
#| export
class BaseErrorType(ABC):
    "Base class of a type error response. This class is not used direclty by developers, but defines the interface common to all."
    def __init__(self, flip_error_response=False):
        self.factor=1
        if flip_error_response:
            self.factor=-1
        self.error_response=None
        self.terminate = False
        
    def __repr__(self):
        if self.error_response == None:
            return f': {self.__class__.__name__} error_response:{self.error_response}'
        return f': {self.__class__.__name__} error_response:{self.error_response * self.factor}'

    def aggregate(self, errors):
        raise NotImplementedError
        
    @abstractmethod
    def __call__(self):
        pass
    
    @abstractmethod
    def reset(self):
        self.error_response=None
        self.terminate = False

    def set_properties(self, properties):
        for property_name, property_value in properties.items():
            setattr(self, property_name, property_value)

    # def set_property(self, property_name, property_value):
    #     exec(f'self.{property_name} = {property_value}')
        
    def get_error_response(self):
        return self.error_response * self.factor
    
    def set_error_response(self, error):
        self.error_response = error * self.factor

    def is_terminated(self):
        return self.terminate    

## RootSumSquaredError

In [ ]:
#| export
class RootSumSquaredError(BaseErrorType):
    "The square root of the sum of the square of the errors."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)
        self.sum=0
        
    def __call__(self, error):
        self.sum+=error*error
        self.error_response=math.sqrt(self.sum)

    def reset(self):
        super().reset()
        self.sum=0
        
    class Factory:
        def create(self, flip_error_response=False): return RootSumSquaredError(flip_error_response=flip_error_response)

## RootMeanSquareError

In [ ]:
#| export
class RootMeanSquareError(BaseErrorType):
    "The square root of the mean of the sum of the square of the errors."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)
        self.reset()
        
    def __call__(self, error):
        self.num+=1
        # Compute the Euclidean norm of the error array
        norms = np.linalg.norm(error)
        self.sum+=norms**2
        self.error_response=np.sqrt(self.sum/self.num)

        # self.sum+=error*error
        # self.error_response=math.sqrt(self.sum/self.num)

    def reset(self):
        super().reset()
        self.sum=0
        self.num=0
        
    class Factory:
        def create(self, flip_error_response=False): return RootMeanSquareError(flip_error_response=flip_error_response)

## SummedError

In [ ]:
#| export
class SummedError(BaseErrorType):
    "Sum of all errors."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)
        self.reset()
        
    def __call__(self, error):
        self.sum+=error
        self.error_response=self.sum

    def reset(self):
        super().reset()
        self.sum=0
        
    class Factory:
        def create(self, flip_error_response=False): return SummedError(flip_error_response=flip_error_response)

## CurrentError

In [ ]:
#| export
class CurrentError(BaseErrorType):
    "The current error, rather than a function of the historical values."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)
    
    def __call__(self, error):
        self.error_response=error

    def reset(self):
        super().reset()

    class Factory:
        def create(self, flip_error_response=False): return CurrentError(flip_error_response=flip_error_response)

## CurrentRMSError

In [ ]:
#| export
class CurrentRMSError(BaseErrorType):
    "The current RMS error, rather than a function of the historical values."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)
    
    def __call__(self, errors):
        self.error_response = math.sqrt(sum(e**2 for e in errors) / len(errors))

    def reset(self):
        super().reset()

    class Factory:
        def create(self, flip_error_response=False): return CurrentRMSError(flip_error_response=flip_error_response)

## SmoothError

In [ ]:
#| export
class SmoothError(BaseErrorType):
    "The exponential smoothed value of the error."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)        
        self.smooth_factor = None
        self.error_response = 0
    
    def set_properties(self, properties):
        if 'smooth_factor' in properties:
            super().set_properties(properties)

    def __call__(self, error):
        self.error_response=smooth(abs(error), self.error_response, self.smooth_factor)
        
    def reset(self):
        self.error_response = 0

    class Factory:
        def create(self, flip_error_response=False): return SmoothError(flip_error_response=flip_error_response)

## MovingSumError

In [ ]:
#| export
class MovingSumError(BaseErrorType):
    "The moving sum of the error."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)        
        self.error_response = 0
        self.history = None
        self.initial = None

    def set_properties(self, properties):
        if 'history' in properties:
            super().set_properties(properties)
    
    def __call__(self, error):
        self.boxcar.append(error)
        self.boxcar.pop(0)
        self.error_response=sum(self.boxcar)
        # self.terminate = ListChecker.check_list_unchanged(self.boxcar)
        

    def reset(self):
        # self.terminate = False
        # self.error_response = 0
        self.boxcar = [self.initial for i in range(1, self.history+1)]
        self.error_response=sum(self.boxcar)

    class Factory:
        def create(self, flip_error_response=False): return MovingSumError(flip_error_response=flip_error_response)

## MovingAverageError

In [ ]:
#| export
class MovingAverageError(BaseErrorType):
    "The moving average of the error."
    def __init__(self, flip_error_response=False):
        super().__init__(flip_error_response=flip_error_response)        
        self.error_response = 0
        self.history = None
        self.initial = None

    def set_properties(self, properties):
        if 'history' in properties:
            super().set_properties(properties)

    def __call__(self, error):
        self.boxcar.append(error)
        self.boxcar.pop(0)
        self.error_response=sum(self.boxcar)/self.history
        
    def reset(self):
        # self.error_response = 0
        self.boxcar = [self.initial for i in range(1, self.history+1)]
        self.error_response=sum(self.boxcar)/self.history

    class Factory:
        def create(self, flip_error_response=False): return MovingAverageError(flip_error_response=flip_error_response)

## ErrorResponseFactory

In [ ]:
#| export
class ErrorResponseFactory:
    factories = {}
    def addResponseFactory(id, errorResponseFactory):
        ErrorResponseFactory.factories.put[id] = errorResponseFactory
    addResponseFactory = staticmethod(addResponseFactory)
    # A Template Method:
    def createErrorResponse(id, flip_error_response=False):
        if not ErrorResponseFactory.factories.__contains__(id):
            ErrorResponseFactory.factories[id] = \
              eval(id + '.Factory()')
        return ErrorResponseFactory.factories[id].create(flip_error_response=flip_error_response)
    createErrorResponse = staticmethod(createErrorResponse)

# Error collection - from each iteration 

## ErrorCollectorFactory

In [ ]:
#| export
class ErrorCollectorFactory:
    factories = {}
    def addCollectorFactory(id, errorCollectorFactory):
        ErrorCollectorFactory.factories.put[id] = errorCollectorFactory
    addCollectorFactory = staticmethod(addCollectorFactory)
    # A Template Method:
    def createErrorCollector(id):
        if not ErrorCollectorFactory.factories.__contains__(id):
            ErrorCollectorFactory.factories[id] = \
              eval(id + '.Factory()')
        return ErrorCollectorFactory.factories[id].create()
    createErrorCollector = staticmethod(createErrorCollector)

## BaseErrorCollector

In [ ]:
#| export
class BaseErrorCollector(ABC):
    "Base class of an error collector. This class is not used direclty by developers, but defines the interface common to all."
    'Parameters:'
    'limit - the limit of valid error response'
    'error_response - the type of error response'
    
    def __init__(self, limit,error_response, min=True):
        self.limit=limit
        self.limit_exceeded=False
        self.error_response=error_response
        self.min=min

        
    def __repr__(self):
        return f'{self.__class__.__name__} limit:{self.limit}, limit_exceeded:{self.limit_exceeded}, {self.error_response.__repr__()}'
        
    def set_min(self, min):
        self.min=min

    def set_limit(self, limit):
        self.limit=limit
        self.limit_exceeded=False

    def set_error_response(self, error_response):
        self.error_response=error_response
    
    def reset(self):
        self.error_response.reset()
        self.limit_exceeded=False

    def set_properties(self, properties):
        for property_name, property_value in properties.items():
            setattr(self, property_name, property_value)

    def error(self):
        return self.error_response.get_error_response()

    def override_value(self):
        if self.limit is not None:
            self.error_response.set_error_response(self.error_response.get_error_response()*abs(self.limit))
    
    def add_error_data(self, data=[]):
        for datum in data:
            self.error_response(datum)

    def add_error_data_array(self, data=[]):
        self.error_response(data)

    def is_terminated(self):            
        if self.limit_exceeded:
            return True
        if self.error_response.is_terminated():
            return True  
        return False

    @classmethod
    def collector(cls, error_response_type, error_collector_type, limit, min=True, properties=None, flip_error_response=False):
        error_response = ErrorResponseFactory.createErrorResponse(error_response_type, flip_error_response=flip_error_response)   
        error_collector = ErrorCollectorFactory.createErrorCollector(error_collector_type)   
        error_collector.set_limit(limit)
        error_collector.set_min(min)
        
        if properties:
            if 'error_response' in properties:
                error_response.set_properties(properties['error_response'])
            if 'error_collector' in properties:     
                error_collector.set_properties(properties['error_collector'])   

        # if properties != None:
        #     for property in properties:
        #         if error_response_type == 'SmoothError' and property[0] == 'smooth_factor':
        #             error_response.set_property(property[0], property[1])
        #             continue
        #         if error_response_type == 'MovingSumError' or error_response_type == 'MovingAverageError':
        #             error_response.set_property(property[0], property[1])
        #             continue
        #         if error_collector_type == 'ReferencedInputsError' and property[0] == 'referenced_inputs':
        #             error_collector.set_reference_properties(property[1])
        #             continue
        
        error_response.reset()
        error_collector.set_error_response(error_response)
        
        return error_collector
    
    def check_limit(self):
        if self.limit is None:
            raise Exception(f': {self.__class__.__name__} requires a limit value')

        if self.min is None:
            raise Exception(f': {self.__class__.__name__} min must be True or False not None')

        if self.min:
            if self.error_response.get_error_response() > self.limit:
                self.limit_exceeded=True
        else:
            if self.error_response.get_error_response() < self.limit:
                self.limit_exceeded=True
        
        return self.limit_exceeded

## TotalError

In [ ]:
#| export
class TotalError(BaseErrorCollector):
    "A class to collect all the errors of the control system run."            
    def __init__(self, limit=None, error_response=None, min=None, **cargs):
        super().__init__(limit, error_response, min)

    def add_data(self, hpct=None):
        for level in range(len(hpct.hierarchy)):
             for col in range(len(hpct.hierarchy[level])):
                node  = hpct.hierarchy[level][col]
                self.add_error_data( [node.get_function("comparator").get_value()])
                if self.check_limit():
#                 if self.error_response.get_error_response() > self.limit:
#                     self.limit_exceeded=True
                    return
    class Factory:
        def create(self): return TotalError()

## TopError

In [ ]:
#| export
class TopError(BaseErrorCollector):
    "A class to collect all the errors of the top-level nodes."            
    def __init__(self, limit=None, error_response=None, min=None, **cargs):
        super().__init__(limit, error_response, min)

    def add_data(self, hpct=None):
        level = len(hpct.hierarchy)-1
            
        for col in range(len(hpct.hierarchy[level])):
            node  = hpct.hierarchy[level][col]
            self.add_error_data( [node.get_function("comparator").get_value()])
            if self.check_limit():
#             if self.error_response.get_error_response() > self.limit:
#                 self.limit_exceeded=True
                return
            
    class Factory:
        def create(self): return TopError()

## InputsError

In [ ]:
#| export
class InputsError(BaseErrorCollector):
    "A class to collect the values of the input values."            
    def __init__(self, limit=None, error_response=None, min=None, **cargs):
        super().__init__(limit, error_response, min)
        self.indexes = []

    def set_properties(self, properties):
        self.indexes = properties.get('indexes', [])

    def add_data(self, hpct=None):
        data = []
        pre = hpct.get_preprocessor()
        for i, func in enumerate(pre[1:], start=1):
            if len(self.indexes) > 0 :
                if (i - 1) in self.indexes:
                    data.append(func.get_value())
            else:
                data.append(func.get_value())
        self.add_error_data_array(data)
        if self.limit and self.check_limit():
            return
        
    class Factory:
        def create(self): return InputsError()

## ReferencedInputsError

In [ ]:
#| export
class ReferencedInputsError(BaseErrorCollector):
    "A class to collect the values of the input values subtracted from reference values."                        
    def __init__(self, limit=None, error_response=None, min=None, **cargs):
        super().__init__(limit, error_response, min)
        self.reference_values=None
        self.input_indexes=None        
        
    def set_properties(self, properties):
        ref_inputs = properties.get('referenced_inputs', {})
        self.input_indexes = ref_inputs.get('indexes', [])
        self.reference_values = ref_inputs.get('refs', [])
        self.weights = ref_inputs.get('weights', [1] * len(self.reference_values))
                

    def add_data(self, hpct=None):
        data = []
        pre = hpct.get_preprocessor()
        for ctr, index in enumerate(self.input_indexes):
            func = pre[index+1] # add 1 as environment is 0
            if isinstance(func, IndexedParameter):
                data.append((self.reference_values[ctr]-func.get_value()) * self.weights[ctr])
            else:
                raise Exception(f'Function {func.get_name()} is not type IndexedParameter.')
        self.add_error_data( data )
        if self.check_limit():
#         if self.error_response.get_error_response() > self.limit:
#             self.limit_exceeded=True
            return

    class Factory:
        def create(self): return ReferencedInputsError()

## RewardError

In [ ]:
#| export
class RewardError(BaseErrorCollector):
    "A class that collects the reward value of the control system run."            
    def __init__(self, limit=None, error_response=None, min=None, **cargs):
        super().__init__(limit, error_response, min)

    def add_data(self, hpct=None):
        data = []
        pre = hpct.get_preprocessor()
        data.append(pre[0].get_reward())
        self.add_error_data( data )
#         if self.check_limit():
#             return
    class Factory:
        def create(self): return RewardError()

## FitnessError

In [ ]:
#| export
class FitnessError(BaseErrorCollector):
    "A class that collects the fitness value of the control system run."            
    def __init__(self, limit=None, error_response=None, min=None, **cargs):
        super().__init__(limit, error_response, min)

    def add_data(self, hpct=None):
        data = []
        pre = hpct.get_preprocessor()
        data.append(pre[0].get_fitness())
        self.add_error_data( data )
        self.check_limit()

    class Factory:
        def create(self): return FitnessError()

# Examples

In [ ]:
rms = RootMeanSquareError()
for i in range(10):
    rms([i])
er = rms.get_error_response()
print(er)
TestCase().assertAlmostEqual(er, 5.338539126015656, places=6)


5.338539126015656


In [ ]:
rsse = RootSumSquaredError()
te = TotalError(error_response=rsse, limit=250,min=True)   
te.add_error_data([1, 2])
print(te)
err=te.error()
print(err)
TestCase().assertAlmostEqual(err, 2.23606797749979, places=6)


TotalError limit:250, limit_exceeded:False, : RootSumSquaredError error_response:2.23606797749979
2.23606797749979


In [ ]:
et = ErrorResponseFactory.createErrorResponse('RootSumSquaredError')   
et(102)
print(et.get_error_response())

iprms = ErrorCollectorFactory.createErrorCollector('TotalError')   
iprms.set_limit(100)
iprms.set_error_response(et)
print(iprms.error())

102.0
102.0


In [ ]:
iprms = BaseErrorCollector.collector( 'RootMeanSquareError','InputsError', 10, flip_error_response=False, min=False)

In [ ]:

time_series_example = np.array([[1, 2, 3],
                                [4, 5, 6],
                                [7, 8, 9],
                                [10, 11, 12],
                                [13, 14, 15]])
for ts in time_series_example:
    iprms.add_error_data(ts)
erms = iprms.error()
print(erms)
print(iprms)
TestCase().assertAlmostEqual(erms, 9.092121131323903, places=6)

9.092121131323903
InputsError limit:10, limit_exceeded:False, : RootMeanSquareError error_response:9.092121131323903


In [ ]:
iprms.reset()
print(iprms)

InputsError limit:10, limit_exceeded:False, : RootMeanSquareError error_response:None


In [ ]:
iprms2 = BaseErrorCollector.collector( 'RootMeanSquareError','InputsError', 10, flip_error_response=False, min=False)
time_series_example2 = np.array([[1, 2, 3],
                                [4, 5, 6],
                                [7, 8, 9],
                                [10, 11, 12],
                                [13, 14, 15]])
for ts in time_series_example2:
    iprms2.add_error_data_array(ts)
erms2 = iprms2.error()
print(erms2)
print(iprms2)
TestCase().assertAlmostEqual(erms2, 15.748015748023622, places=6)

15.748015748023622
InputsError limit:10, limit_exceeded:False, : RootMeanSquareError error_response:15.748015748023622


In [ ]:
iprms1 = BaseErrorCollector.collector( 'RootMeanSquareError','InputsError', 10, flip_error_response=False, min=False)
iprms1.add_error_data([3])
iprms1.add_error_data([5])
erms1 = iprms1.error()
print(erms1)
print(iprms1)
TestCase().assertAlmostEqual(erms1, 4.123105625617661, places=6)

4.123105625617661
InputsError limit:10, limit_exceeded:False, : RootMeanSquareError error_response:4.123105625617661


In [ ]:
time_series = np.array([[1, 2, 3],
                                [4, 5, 6],
                                [7, 8, 9],
                                [10, 11, 12],
                                [13, 14, 15]])
norms = np.linalg.norm(time_series, axis=1)
print(norms)

[ 3.74165739  8.77496439 13.92838828 19.10497317 24.2899156 ]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()